In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
import os 
from dotenv import load_dotenv
from time import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
#loader = DirectoryLoader('PDF_Testing', glob="./*.pdf", loader_cls=PyPDFLoader)
loader = PyPDFLoader('How AI is expanding art history.pdf')
documents = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [4]:
len(texts)

16

In [5]:
texts[5]

Document(page_content='unease or sensuality. \nUsing AI methods, computers can analyse \nsuch poses in tens of thousands of portraits \nin as little as an hour, much quicker than an \nart scholar can. Deep neural networks — \nmachine-learning systems that mimic biolog -\nical neural networks in brains — can detect the \nlocations of key points, such as the tip of the \nnose or the corners of the eyes, in a painting. \nThey then accurately infer the angles of a sub -\nject’s pose around three perpendicular axes \nfor realistic and highly stylized portraits.\nFor example, earlier this year, researchers \nused deep neural networks to analyse poses \nand gender across more than 20,000 portraits, \nspanning a wide range of periods and styles, \nto help art scholars group works by era and art \nmovement. There were some surprises — the \ntilts of faces and bodies in self-portraits vary \nwith the stance of the artist, and the algorithms \ncould tell whether the self-portraitists were', metad

In [6]:
_ = load_dotenv()

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

llm=HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta", 
    model_kwargs={"temperature":0.2, "max_length":256},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
    )

In [7]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="BAAI/bge-base-en-v1.5",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [8]:
%%time
persist_directory = 'db_HuggingFace'

embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

CPU times: total: 1.2 s
Wall time: 3.94 s


In [9]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
docs = retriever.get_relevant_documents("What is paranoia?")

In [10]:
len(docs)

2

In [11]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [12]:
def process_llm_response(qa_chain, query):
    print(f"Query: {query}\n")
    time_1 = time()
    llm_response = qa_chain(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print(f"\nResult:", llm_response['result'])
    print(f"\nmetadata:", llm_response['source_documents'][0].metadata)

In [13]:
query = "How does AI affect Art?"
process_llm_response(qa_chain, query)

Query: How does AI affect Art?

Inference time: 1.556 sec.

Result:  In the study of art, AI is revealing new insights into color and style by analyzing large datasets of artworks. AI-driven tools are helping artists understand the science behind their creative choices, allowing them to make more informed decisions in their work. However, to fully harness the power of AI in art, we will need the same foundations as other fields, such as data privacy and transparency in algorithmic decision-making. Overall, AI is enhancing the creative process in art by providing new perspectives and tools for exploration.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [14]:
# break it down
query = "How does AI mimic human artwork?"
process_llm_response(qa_chain, query)

Query: How does AI mimic human artwork?

Inference time: 2.136 sec.

Result:  AI-driven tools are revealing how artists' understanding of the science of color and style contributes to their unique artistic style, allowing for the creation of AI-generated artwork that mimics the style of human artists. This is made possible by the same foundations required for the full power of AI in the study of art.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [15]:
query = "How do they measure Momentary social anxiety?"
process_llm_response(qa_chain, query)

Query: How do they measure Momentary social anxiety?

Inference time: 0.579 sec.

Result:  The study assessed momentary social anxiety using three specific items suggested in a previous study. Unfortunately, the specific items are not provided in this context, so the exact measurement method is not clear. However, it can be inferred that these items likely focus on measuring the intensity and duration of social anxiety in specific situations or interactions. Without further information, it is impossible to provide a more detailed answer.

metadata: {'page': 5, 'source': 'PDF_file\\s41598-023-47912-0.pdf'}


In [16]:
query = "Please give me an example of using AI to produce an artwork"
process_llm_response(qa_chain, query)

Query: Please give me an example of using AI to produce an artwork

Inference time: 5.329 sec.

Result:  One example of using AI to produce an artwork is the image generator Dall-E, which has been trained on a nearly terabyte dataset of images and text. By inputting a description or prompt, such as "a surreal landscape with a melting clock tower and floating flowers," Dall-E can generate a unique and original image based on that description. While the resulting images may not be entirely realistic or photorealistic, they can be visually striking and surreal, showcasing the potential of AI to create entirely new and imaginative works of art.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [17]:
query = "How does machine learning and deep neural network help developing AI in art field?"
process_llm_response(qa_chain, query)

Query: How does machine learning and deep neural network help developing AI in art field?

Inference time: 0.596 sec.

Result:  Machine learning and deep neural networks are essential foundations for realizing the full power of AI in the study of art. These technologies allow AI-driven tools to analyze brush strokes, color, and style in fine-art paintings and drawings, revealing new insights and shedding light on these works of art. By providing the same foundations as other fields, machine learning and deep neural networks are helping to develop AI in the art field, unlocking new possibilities for understanding and appreciating art.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [18]:
query = "What is By-eye art analysis?"
process_llm_response(qa_chain, query)

Query: What is By-eye art analysis?

Inference time: 0.596 sec.

Result:  By-eye art analysis refers to the traditional method of analyzing and examining artworks using human vision and perception. It involves looking at the artwork with the naked eye and making observations and interpretations based on visual cues and details. Computer-assisted connoisseurship, on the other hand, uses technology and algorithms to automate or enhance this process, either by replicating the human visual analysis or by providing new insights and perspectives beyond what is possible through normal human perception.

metadata: {'page': 0, 'source': 'How AI is expanding art history.pdf'}


In [19]:
query = "Other than produce Artwork, what else can AI use in Art study?"
process_llm_response(qa_chain, query)

Query: Other than produce Artwork, what else can AI use in Art study?

Inference time: 0.607 sec.

Result:  AI can be used to analyze and interpret art, as well as to assist in conservation efforts by identifying areas of damage or deterioration in paintings and sculptures. It can also be used to help curate exhibitions by suggesting connections between different works of art based on their stylistic features. Additionally, AI can be used to assist in art education by providing personalized feedback to students on their artistic abilities and suggesting areas for improvement.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [20]:
query = "Give me an example of AI help recovering old art pieces"
process_llm_response(qa_chain, query)

Query: Give me an example of AI help recovering old art pieces

Inference time: 4.7 sec.

Result:  One potential application of AI in the study of art is the restoration of damaged or deteriorated artworks. AI algorithms can analyze the original image and predict what the missing or damaged parts might have looked like, allowing for a more accurate and less invasive restoration process. This technology has already been used to restore ancient manuscripts and frescoes, and as the availability of high-quality data increases, we can expect more advanced and sophisticated AI tools to emerge in this field.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [21]:
query = "What is the future of AI in art study?"
process_llm_response(qa_chain, query)

Query: What is the future of AI in art study?

Inference time: 0.6 sec.

Result:  In order to fully harness the potential of AI in art scholarship, we will need to establish the same foundational principles as other fields. While the study of art has evolved over centuries, the introduction of computational tools and AI presents a new frontier for art scholarship. While there is still much work to be done, the potential for AI in art study is promising.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [22]:
query = "Is there any challenge that AI facing in art study?"
process_llm_response(qa_chain, query)

Query: Is there any challenge that AI facing in art study?

Inference time: 0.6 sec.

Result:  Yes, in order to fully harness the power of AI in the study of art, we need to establish the same foundations as other fields of perception and introduce new approaches and classes of questions to art scholarship. This presents a challenge as it requires a deep understanding of both the technical aspects of AI and the nuances of art history and criticism. Additionally, there is a need to address issues of data availability, privacy, and intellectual property in the context of AI and art.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [23]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x1ea592fba90>)

In [24]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:
